# Project 2

## Topic : Stereo reconstruction and Non-linear optimization

#### Instructions
<ul>
    <li> The second project of the course is designed to get you familiar with stereo reconstruction, and non-linear optimization </li>
    <li> Use python for this project. PILLOW and OpenCV are permitted for image I/O. </li>
    <li> Submit this notebook as a zipped file on moodle. The format should be $<$team_id$>$_$<$team_ name$>$.zip. Both members have to submit this zip file. </li>
    <li> A seperate report is not needed if you're coding in the notebook itself. Please provide adequate descriptions of the approaches you've taken. Also mention work distribution for the two members. </li>
    <li> Refer to the late day policy. Start early </li> 
    <li> Download data from here: https://iiitaphyd-my.sharepoint.com/:f:/g/personal/aryan_sakaria_students_iiit_ac_in/Er5C7351IAlFsvwHUesFeSQBQtlSiAS7AORSEJT2qH_8_w?e=ol98k9  </li>
</ul>

----
### PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds fromeach disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    

In [1]:
#import libraries:
import numpy as np 
from sklearn.preprocessing import normalize #normalizing gives better results. Experiment with this
import cv2
import open3d as o3d

In [2]:
def read_transformations(filename='./mr2020_project_data/poses.txt'):
    f = open(filename, 'r')
    lines = f.readlines()
    transformation_list = []
    for i in range(len(lines)):
        transformation_list_temp = lines[i].split()
        temp_rot = [] 
        temp_rot.append( (transformation_list_temp[0:4] ) ) 
        temp_rot.append( (transformation_list_temp[4:8]  ) ) 
        temp_rot.append( (transformation_list_temp[8:12]  ) ) 
        transformation_list.append(temp_rot)
    return transformation_list
tr_list=np.array(read_transformations())
tr_list=tr_list.flatten()
tr_mat_list = list(map(float, tr_list)) 
tr_mat_list=np.array(tr_mat_list)
tr_mat_list=tr_mat_list.reshape(21,3,4)
print(tr_list.shape,tr_mat_list.shape)

(252,) (21, 3, 4)


Provide explanation in this cell: 


In [3]:
#your code here
K = np.array([[7.070912e+02, 0.000000e+00, 6.018873e+02],
             [0.000000e+00, 7.070912e+02, 1.831104e+02],
             [0.000000e+00, 0.000000e+00, 1.000000e+00]])
f=K[0][0]
b=0.53790448812
pcd_gl=o3d.geometry.PointCloud()
pcd = o3d.geometry.PointCloud()
pcd_list=[]
for i in range(21):
    img_1 = cv2.imread('./mr2020_project_data/img2/00000004'+str(i+60)+'.png')
    img_2 = cv2.imread('./mr2020_project_data/img3/00000004'+str(i+60)+'.png')
    
    win_size = 5
    blockSize= 5
    min_disp = -39
    num_disp = 144 
    
    stereo = cv2.StereoSGBM_create(minDisparity= min_disp,
           numDisparities = num_disp,
           blockSize = 5,
           uniquenessRatio = 10,
           speckleWindowSize = 150,
           speckleRange = 32,
           disp12MaxDiff = 1,
           P1 = 8*3*blockSize**2,
           P2 =32*3*blockSize**2,
           preFilterCap=63) 
        
    disparity= stereo.compute(img_1, img_2).astype(np.float32) / 64
    disparity = (disparity-min_disp)/num_disp
    Dim = disparity.shape
    Q = np.array([[ 1,  0,  0, -Dim[1]/2],
                  [ 0,  -1,  0, Dim[0]/2],
                  [ 0,  0,  0, f],
                  [ 0,  0,  1/b, 0]])
    disparity_pts=[]
    for p in range(Dim[0]):
        for q in range(Dim[1]):
            disparity_pts.append([q,p,disparity[p,q],1])
    disparity_pts = np.array(disparity_pts)
    cam_pts=[np.matmul(Q,m) for m in disparity_pts]
    cam_pts=np.array([c/c[3]for c in cam_pts])
    colors = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)
    mask = disparity >= disparity.min()
    colors = colors[mask]
    colors=colors/255
    world_pts=np.matmul(tr_mat_list[i],cam_pts.T)
    world_pts[0]=-world_pts[0]
        
    pcd.points = o3d.utility.Vector3dVector(world_pts.T)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    pcd_list.append(pcd)
    pcd_gl=pcd_gl+pcd
    
pts=np.asarray(pcd_gl.points)
clrs=np.asarray(pcd_gl.colors)
mask = ((-1500 <= pts[:,1]) & (pts[:,1] < 1500) &
         (-1500 <= pts[:,2]) & (pts[:,2] < 1500) &
        (-1500 <= pts[:,0]) & (pts[:,0] < 1500))
pts=pts[mask]
print(np.shape(pts))
clrs=clrs[mask]
pcd_gl.points=o3d.utility.Vector3dVector(pts)
pcd_gl.colors=o3d.utility.Vector3dVector(clrs)

o3d.visualization.draw_geometries([pcd_gl])

o3d.io.write_point_cloud("Out.pcd", pcd_gl)

(7021724, 3)


----
### PART 2: Motion estimation using iterative PnP

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>

In [ ]:
def read_pcd(file_name):
    pcd = o3d.io.read_point_cloud(file_name) # Read the point cloud
    points = np.asarray(pcd.points)
    pcd_points=np.hstack((points,np.ones((len(points),1))))
    return pcd_points

X_gt = read_pcd("Out.pcd")
X_gt = X_gt[0::10,:]
T_gt = np.eye(4,4)
T_gt[:3,:4] = T[0]
T_gt = np.linalg.inv(T_gt)
P_gt = K@T_gt[:3,:4]
x_gt = (P_gt@X_gt.T).T

def DLT(x,X):
    x[:,0] = x[:,0]/x[:,2];
    x[:,1] = x[:,1]/x[:,2];
    L = len(X);
    zeros = np.zeros((L,4))
    A = (np.vstack((x[:,0]*X[:,0],np.vstack((x[:,0]*X[:,1],np.vstack((x[:,0]*X[:,2],x[:,0]*X[:,3]))))))).T
    B = (np.vstack((x[:,1]*X[:,0],np.vstack((x[:,1]*X[:,1],np.vstack((x[:,1]*X[:,2],x[:,1]*X[:,3]))))))).T
    Ax = np.hstack((-X,np.hstack((zeros,A))));
    Ay = np.hstack((zeros,np.hstack((-X,B))));    
    M = np.vstack((Ax,Ay))
    U,D,VT = np.linalg.svd(M);
    P = np.array(VT[-1]).reshape(3,4);
    P = P/P[-1,-1]
    return P


def residual(x,x_gt):
    A = x_gt[:,0]/x_gt[:,2] - x[:,0]/x[:,2]
    B = x_gt[:,1]/x_gt[:,2] - x[:,1]/x[:,2]
    res = (np.hstack((A,B))).reshape(-1,1)
    return res

def Jacobian(x,X):
    L = len(X);
    X[:,0] = X[:,0]/x[:,2]
    X[:,1] = X[:,1]/x[:,2]
    X[:,2] = X[:,2]/x[:,2]
    X[:,3] = X[:,3]/x[:,2]
    x[:,0] = x[:,0]/x[:,2]
    x[:,1] = x[:,1]/x[:,2]
    zeros = np.zeros((L,4))
    A = (np.vstack((x[:,0]*X[:,0],np.vstack((x[:,0]*X[:,1],np.vstack((x[:,0]*X[:,2],x[:,0]*X[:,3]))))))).T
    B = (np.vstack((x[:,1]*X[:,0],np.vstack((x[:,1]*X[:,1],np.vstack((x[:,1]*X[:,2],x[:,1]*X[:,3]))))))).T
    J_x = np.hstack((-X,np.hstack((zeros,A))));
    J_y = np.hstack((zeros,np.hstack((-X,B))));
    J = np.vstack((J_x,J_y))
    return J


def Gauss_newton(P,x_gt,X,N,tol): 
    Cost = np.zeros(N)
    for i in range(N):
        x = (P@X.T).T
        res = residual(x,x_gt) 
        Cost[i] = (res.T@res/2/len(X)).item(0);
        if(i==0):
            print("\nInitial Cost (in terms of MSE) = ",Cost[0])
        J = Jacobian(x,X);
        H = J.T@J;
        update = np.linalg.inv(H)@J.T@res;
        P = (P.reshape(12,1) - update).reshape(3,4);
        if(np.linalg.norm(update)<tol):
            print("\nGN has converged")
            break;
    return P, Cost, i

In [ ]:

P_dlt = DLT(x_gt[:10,:],X_gt[:10,:]);

print("P_GT = ")
print(P_gt)
print("\nT_GT = ")
print(np.linalg.inv(K)@P_gt)

print("\nInitialisation using P_DLT = ")
print(P_dlt)

P_op, Cost, it_conv = Gauss_newton(P_dlt,x_gt,X_gt,1000,1e-4)
P_op = P_gt[-1,-1]*P_op/P_op[-1,-1] 

print("\nFinal Cost (in terms of MSE) = ",Cost[it_conv])
print("\niterations to converge = ",it_conv+1)
print("\nP_Output = ")
print(P_op)
print("\nT_Output = ")
print(np.linalg.inv(K)@P_op)

plt.figure(figsize=(16,7));
plt.plot(range(it_conv+1), Cost[0:it_conv+1]);
plt.title('Cost (in terms of MSE) vs Number of Iterations')
plt.xlabel('Number of Iterations')
plt.ylabel('Cost (in terms of MSE)')